In [1]:
'''
DB(Mysql)に保存した文章データから、形態素解析するサンプル。
取り出した語句についてWordCloudで可視化までする
'''

import re
import pickle
from janome.tokenizer import Tokenizer
import mysql.connector
from wordcloud import WordCloud


#文章データを読み込みます。
#この例では、mysqlから読み込み元のデータを持って来ています。
conn = mysql.connector.connect(
    host = 'localhost',
    port =  3306,
    user =  'root',
    password =  "",
    database = "BlogResearch",
)
cur = conn.cursor()
cur.execute('SELECT diary_data  from t_diary_logs where  id IN(3525)')
datas = cur.fetchall()

seperator = "。"
mixi_diary_origin = ""
for row in datas:
    data =   re.sub("[｜ 　「」\n]", "", row[0]) # | と全角半角スペース、「」と改行の削除
    mixi_diary_origin += data

mixi_diary_list = mixi_diary_origin.split(seperator)  # セパレーターを使って文章をリストに分割する
mixi_diary_list = [x+seperator for x in mixi_diary_list]  # 文章の最後に。を追加

#この時点でデータの準備が終わりです
#ここから形態素分析に入ります
t = Tokenizer()

mixi_diary_words = []  #形態素分析したあとに出てきた語句を格納するリスト(この例では、名詞、形容詞のみの語句を取っています)
for sentence in mixi_diary_list:
    
    tokens = t.tokenize(sentence)
    words = []
    for token in tokens:
        # 品詞を取り出し
        partOfSpeech = token.part_of_speech.split(',')[0]
 
        if partOfSpeech == u'名詞'  or  partOfSpeech == u'形容詞' :
            #print (token.surface)
            words.append(token.surface)
    
    if len(words) > 0:
        mixi_diary_words.extend(words)
    


In [2]:
#ここからWordCloudの処理
#mixi_diary_wordsは、WordCloudで使えるように2次元の配列(行列)になっているので、1次元の配列(ベクトル)に変換する
text = ""
for x in mixi_diary_words:
    text = text + x + " "

wordcloud = WordCloud(background_color="white",
    font_path="/System/Library/Fonts/ヒラギノ明朝 ProN.ttc",
    width=800,height=600).generate(text)

#画像ファイルに保存する
wordcloud.to_file("./wordcloud_mixi.png")